In [1]:
!pip install -qqq autogen groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.1/350.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

GROQ_API_KEY=userdata.get('GROQ_API_KEY')

In [3]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

temp_dir=tempfile.TemporaryDirectory()
config_list = [
    {
        # Let's choose the Llama 3 model
        "model": "llama-3.1-70b-versatile", #llama-3.2-90b-text-preview

        # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
        "api_key": GROQ_API_KEY,
        # We specify the API Type as 'groq' so it uses the Groq client class
        "api_type": "groq",
    }
]

executer=LocalCommandLineCodeExecutor(timeout=20,work_dir=temp_dir.name)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [23]:

code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
You need to calculate:

    1- The number of packs of each ingredient to buy, ensuring total cost is within the budget.
    2- The number of units of each product to sell, maximizing profit above inventory costs.

Constraints:

    1- Budget Constraint:
        The total cost of ingredients must not exceed the budget:

    buying_price1 * pack_count1 + buying_price2 * pack_count2 + ... + buying_price34 * pack_count34 <= Budget=2500

    2- Profit Constraint:

    Profit after deducting inventory costs must be positive:

        ((sell_price1 * number_of_selling1 + sell_price2 * number_of_selling2 + ... + sell_price40 * number_of_selling40)
         - (buying_price1 * pack_count1 + buying_price2 * pack_count2 + ... + buying_price34 * pack_count34)) > 0

    3- Minimum Pack Count:
        Each ingredient must have a minimum pack count of 1 to ensure stock availability.

Dataset Structure:

You will work with a JSON dataset with 40 products, each containing the following structure:

{
  "products": [
    {
      "name": "Espresso",
      "sell_price": 1.40,
      "sale_ratio": "10%",
      "ingredients": [
        {"name": "Espresso beans", "quantity": 22, "unit": "grams", "buying_price": 17.3, "buying_unit": "1000g/pack"},
        {"name": "Water", "quantity": 60, "unit": "ml", "buying_price": 0.5, "buying_unit": "1500ml/bottle"}
      ]
    },
    {
      "name": "Americano",
      "sell_price": 1.50,
      "sale_ratio": "4%",
      "ingredients": [
        {"name": "Espresso beans", "quantity": 22, "unit": "grams", "buying_price": 17.3, "buying_unit": "1000g/pack"},
        {"name": "Water", "quantity": 180, "unit": "ml", "buying_price": 0.5, "buying_unit": "1500ml/bottle"}
      ]
    }
  ]
}

Each product has:

    sell_price: The price at which the product is sold.
    sale_ratio: Expected sales percentage in relation to other products.
    ingredients: List of ingredients required, with each ingredient containing:
        quantity: Amount needed per unit of product.
        buying_price: Price per unit pack.
        buying_unit: Pack size for the ingredient.

Instructions:

    1- Calculate Ingredient Packs:
        For each of the 34 ingredients, determine the minimum number of packs required based on the products’ demands and the pack sizes.
        Just use buying_price for calculating the pack count.
        Ensure that the total ingredient cost does not exceed the specified budget.
        Each ingredient’s pack count must be at least 1.

    2- Calculate Product Quantities:
        Based on the sale_ratio and sell_price, calculate the optimal quantity of each product to sell to maximize profit.
        Ensure that the total profit minus inventory cost is positive, as per the profit constraint.
        You do not need sale ratio in this part.

    3- Output Format:
        Provide the result as a JSON output with ingredient pack counts and product sale quantities in the following format:

        {
          "ingredients": {
            "Ingredient1": PackCount1,
            "Ingredient2": PackCount2,
            ...
            "Ingredient34": PackCount34
          },
          "products": {
            "Product1": SellCount1,
            "Product2": SellCount2,
            ...
            "Product40": SellCount40
          }
        }

        Include all 34 ingredients with their calculated pack counts (no omissions).
        Include all 40 products with their calculated quantities to sell.

    Additional Guidelines:
        * Ensure that pack counts and sale counts are integers with no zero values.
        Distribute the budget efficiently to avoid purchasing excess packs where unnecessary.

Example Task Flow:

    Read the JSON file from the given path.
    Calculate the required pack counts for each ingredient, based on buying_price and adhering to budget.
    For calculating the required pack counts, find the best numbers to make the output close to budget, which is 2500.
    Calculate the selling quantities for each product, ensuring profitability based on given sell prices.
    The numbers should be rationale. You can see the sale_ratio. If the sale ration of a product is high it must be sold more.
    Ensure that pack counts and sale counts are integers with no zero values.
    Output the optimized inventory and sales plan in JSON format.

Execution Requirements:

    Provide the complete code in a single Python code block for direct execution.
    Verify the output format matches the required JSON structure precisely.
    Do not make any assumptions or fabricate values; use only the data provided in the JSON.
    Output the final JSON file with the calculated pack and sale counts.
"""

In [24]:

code_writer_agent=ConversableAgent(
    'code_writer',
    llm_config={"config_list": config_list},
    system_message=code_writer_system_message,
    code_execution_config=False

)

code_executer_agent=ConversableAgent(
    'code_executer',
    llm_config=False,
    code_execution_config={'executor':executer},
    human_input_mode='ALWAYS',
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"]
)


In [25]:
chat_result = code_executer_agent.initiate_chat(
    code_writer_agent,
    message=f"Here is the path: /content/Full_Dataset.json, after that you done your job  save the JSON to a file named 'inventory_product.json",

)

code_executer (to code_writer):

Here is the path: /content/Full_Dataset.json, after that you done your job  save the JSON to a file named 'inventory_product.json

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executer):

To calculate the required pack counts for each ingredient, and the selling quantities for each product, I will use a Python script that reads the JSON file, performs the necessary calculations, and outputs the optimized inventory and sales plan in JSON format.

The script will ensure that pack counts and sale counts are integers with no zero values. It will also try to utilize the entire budget for calculating package numbers.

Here is the Python code:

```python
# filename: inventory_optimizer.py
import json
import math

# Read the JSON file from the given path
def read_json_file(path):
  with open(path, 'r') as file:
    data = json.load(file)
    return data

# Calculate the requi

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executer (to code_writer):

exitcode: 0 (execution succeeded)
Code output: {
  "ingredients": {
    "Espresso beans": 3,
    "Water": 1,
    "Espresso": 19,
    "Milk": 274,
    "Chocolate syrup": 7,
    "Ice-cream": 57,
    "Caramel sauce": 2,
    "Black tea loose leaf": 2,
    "Green tea loose leaf": 1,
    "Dried hibiscus flowers": 1,
    "Dried mint leaf": 1,
    "Honey": 12,
    "Dried ginger root": 1,
    "All-purpose flour": 66,
    "Sugar": 125,
    "Cocoa powder": 11,
    "Baking powder": 2,
    "Baking soda": 4,
    "Salt": 2,
    "Eggs": 1,
    "Vegetable oil": 44,
    "Vanilla extract": 5,
    "Butter": 24,
    "Lemon juice": 27,
    "Ground cinnamon": 1,
    "Ground ginger": 1,
    "Grated carrots": 19,

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.


In [26]:
import os

print(os.listdir(temp_dir.name))

['tmp_code_f8cc38aaced5688579478d72b35f431f.py', 'inventory_product.json', 'read_json.py', 'inventory_optimizer.py', 'calculate_inventory.py']


In [27]:
import json

file=os.path.join(temp_dir.name,'inventory_product.json')
with open(file, 'r') as f:
    data = json.load(f)

In [28]:
data

{'ingredients': {'Espresso beans': 3,
  'Water': 1,
  'Espresso': 19,
  'Milk': 274,
  'Chocolate syrup': 7,
  'Ice-cream': 57,
  'Caramel sauce': 2,
  'Black tea loose leaf': 2,
  'Green tea loose leaf': 1,
  'Dried hibiscus flowers': 1,
  'Dried mint leaf': 1,
  'Honey': 12,
  'Dried ginger root': 1,
  'All-purpose flour': 66,
  'Sugar': 125,
  'Cocoa powder': 11,
  'Baking powder': 2,
  'Baking soda': 4,
  'Salt': 2,
  'Eggs': 1,
  'Vegetable oil': 44,
  'Vanilla extract': 5,
  'Butter': 24,
  'Lemon juice': 27,
  'Ground cinnamon': 1,
  'Ground ginger': 1,
  'Grated carrots': 19,
  'Cream': 19,
  'Strawberries': 45,
  'Graham cracker crumbs': 10,
  'Cream cheese': 1,
  'Banana': 1,
  'Yogurt': 12,
  'Mango': 16,
  'Peanut butter': 2},
 'products': {'Espresso': 4,
  'Americano': 4,
  'Latte': 6,
  'Cappuccino': 6,
  'Macchiato': 5,
  'Mocha': 8,
  'Flat white': 6,
  'Affogato': 7,
  'Caramel macchiato': 10,
  'Misto': 5,
  'Cortado': 5,
  'Black tea': 4,
  'Green tea': 4,
  'Sour te

In [29]:
with open('inventory__profit_optimization.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)